In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from submodular_monotone_net import SubmodularMonotoneNet
import torch
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
monotone_network = SubmodularMonotoneNet(5, 0.5, 1, [1, 100, 100, 1])

/Users/devarora/ml/submodular_nn/increasing_convex_net.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(
/Users/devarora/ml/submodular_nn/increasing_convex_net.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(


In [41]:
monotone_network.clamp_weights()
# monotone_network.activ = lambda x: torch.min(torch.zeros_like(x), x)

monotone_network

# with torch.no_grad():
    # for layer in monotone_network.phi.bs:
      #   layer.data = torch.clamp(layer.data, 0, torch.inf)
    
    # for i in range(monotone_network.layers):
    #     monotone_network.m[i].bias.clamp_(0, torch.inf)

SubmodularMonotoneNet(
  (phi): IncreasingConvexNet(
    (Ws): ParameterList(
        (0): Parameter containing: [torch.float32 of size 100x1]
        (1): Parameter containing: [torch.float32 of size 100x100]
        (2): Parameter containing: [torch.float32 of size 1x100]
    )
    (bs): ParameterList(
        (0): Parameter containing: [torch.float32 of size 100]
        (1): Parameter containing: [torch.float32 of size 100]
        (2): Parameter containing: [torch.float32 of size 1]
    )
  )
  (m): ModuleList(
    (0-4): 5 x Linear(in_features=1, out_features=1, bias=True)
  )
)

In [42]:
class TParams:
    def __init__(self, lr=0.001, b1=None, b2=None, iters=int(1e3)):
        self.lr = lr
        self.b1 = b1
        self.b2 = b2
        self.iters = iters

    def __str__(self):
        return json.dumps(
            {"lr": self.lr, "b1": self.b1, "b2": self.b2, "iters": self.iters}
        )

In [43]:
def loss(net, x, y):
    return torch.sum((net.forward(x) - y) ** 2)

In [57]:
def train(net, fit_fn, a, b, dx, tparams, verbose=True):
    """
    Train the net to minimize a least squares objective between itself and the given fit_fn on the interval [a, b]
    by fitting regularly sampled points [a, a+dx, a+2dx, ..., b].

    fit_fn - (np.ndarray) -> (np.ndarray): map an input tensor to an output tensor
    a - (torch.float32): beginning of interval
    b - (torch.float32): end of interval
    dx - (torch.float32): step size
    tparams - TParams: parameters to use for training
    """
    x = torch.tensor(
        torch.arange(a, b, dx), dtype=torch.float32
    ).view([-1, 1])
    y = fit_fn(x)

    opt = torch.optim.Adam(
        params=net.parameters(), lr=tparams.lr
    )

    if verbose:
        rng = trange(tparams.iters)
    else:
        rng = range(tparams.iters)

    for itr in rng:
        opt.zero_grad()
        loss_this_itr = loss(net, x, y)
        
        loss_this_itr.backward()
        if (itr % 100 == 0):
            if (not verbose): print(f"Iteration: {itr + 1}")
            print(loss_this_itr)
        opt.step()
        net.clamp_weights()


In [53]:
def plot(net, fit_fn, a, b, dx):
    x = torch.arange(a, b, dx)
    x_pt = torch.tensor(x, dtype=torch.float32).view([-1, 1])
    y_true = fit_fn(x)
    y_est = net.forward(x_pt).cpu().detach().numpy()

    plt.plot(x, y_true, label="True")
    plt.plot(x, y_est, label="Estimate")
    plt.show()


In [58]:
# fit_fn = lambda x : torch.ones_like(x) - torch.exp(-x)
fit_fn = lambda x : -(x ** 2)
a = -10
b = 10
dx = 0.1

tparams = TParams(iters=int(5e3))

In [59]:
train(monotone_network, fit_fn, a, b, dx, tparams, verbose=False)

/var/folders/6z/xg9c2xxn1lg34383kjmcgxfw0000gn/T/ipykernel_40945/197256121.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(


TypeError: loss() takes 2 positional arguments but 3 were given

In [ ]:
plot(monotone_network, fit_fn, a, b, dx)